This notebook can be run to create the master table for the number of months indicated in the variable num_months. The tables that are created have the format described in Section 2.4 of the thesis.

This notebook is meant to be run from Google Colab. The following cell connects the notebook with the drive account which has access to the data, which is located at the path.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/TFG ICO/Notebooks/Tables/'
num_months = 12

In [ ]:
import numpy as np
import pandas as pd
import pickle

## **Part I:** Filtering and transforming

Load the original tables *pacients*, *problemes* and *facturacions*, which contain information about patients, diagnoses and medications, respectively.

In [ ]:
pacients = pd.read_csv(path + 'pacients.csv')
problemes = pd.read_csv(path + 'problemes.csv')
facturacions = pd.read_csv(path + 'facturacions.csv')

Select the columns which will be used:
- *ID*, *sexe*, *dnaix*, *sortida* and *situacio* in **pacients**.
- *ID*, *cod*, *dat* and *dbaixa* in **problemes** (all the columns in the table)
- *ID*, *cod*, *dat* and *env* in **facturacions** (all the columns in the table)

In [ ]:
pacients = pacients[['ID', 'sexe', 'dnaix', 'sortida', 'situacio']]
pacients.head()

In [ ]:
problemes.head()

In [ ]:
facturacions.head()

Change all the date columns in the dataframes to datetime datatype.

In [ ]:
pacients.dnaix = pd.to_datetime(pacients.dnaix)
pacients.sortida = pd.to_datetime(pacients.sortida)
problemes.dat = pd.to_datetime(problemes.dat)
problemes.dbaixa = pd.to_datetime(problemes.dbaixa)
facturacions.dat = pd.to_datetime(facturacions.dat)

Truncate diagnosis code. Consider only the main code (according to ICD-10).

In [ ]:
problemes.cod = problemes.cod.apply(lambda x: x[:3])

Select only rows corresponding to diagnosis/medication codes recommended by the experts in the medical field: **Miquel Angel Pujana** and **Conxa Violan**

In [ ]:
diagnoses_codes = [
    'C50', 'E11', 'I10', 'J45', 'T78', 'J44', 'L20', 'M06', 'T45', 'J30', 'J00', 'L50', 'L30', 'H10', 'F33'
]
problemes = problemes[problemes.cod.isin(diagnoses_codes)]

In [ ]:
medication_codes = [
    'A02BA02', 'A02BA03', 'A02BA07', 'A02BA53', 'L02AE01', 'L02AE02', 'L02AE03', 'L02AE04', 'L02AE05',
    'L02AE51', 'L02BA01', 'L02BA02', 'L02BA03', 'L02BG01', 'L02BG02', 'L02BG03', 'L02BG04', 'L02BG05',
    'L02BG06', 'R06AX01', 'R06AX02', 'R06AX03', 'R06AX04', 'R06AX05', 'R06AX07', 'R06AX08', 'R06AX09',
    'R06AX11', 'R06AX12', 'R06AX13', 'R06AX15', 'R06AX16', 'R06AX17', 'R06AX18', 'R06AX19', 'R06AX21',
    'R06AX22', 'R06AX23', 'R06AX24', 'R06AX25', 'R06AX26', 'R06AX27', 'R06AX28', 'R06AX29', 'R06AX31',
    'R06AX32', 'R06AX53', 'R06AX58'
]
facturacions = facturacions[facturacions.cod.isin(medication_codes)]

Convert the categorical variable *sexe* to boolean. As Machine Learning algorithms work better with float values, the new column *is_woman* will take values 0.0 and 1.0

In [ ]:
pacients['is_woman'] = (pacients.sexe == 'D').astype(float)
pacients.drop(columns='sexe', inplace=True)
pacients.head()

Select only data from patients with their first breast cancer diagnosis (code C50) between 01-01-2012 and 31-12-2016 (in fact, just later than 01-01-2012)

In [ ]:
unique_id_before = len(problemes[problemes.cod == 'C50']['ID'].unique())
print('patients with C50 in the original problems table: '+str(unique_id_before))

In [ ]:
df_bcancer = problemes[problemes.cod == 'C50'].groupby('ID')['dat'].min().reset_index()
df_bcancer = df_bcancer[df_bcancer.dat >= '2012-01-01']
bcancer_ids = df_bcancer['ID'].unique()

pacients = pacients[pacients.ID.isin(bcancer_ids)]
problemes = problemes[problemes.ID.isin(bcancer_ids)]
facturacions = facturacions[facturacions.ID.isin(bcancer_ids)]

In [ ]:
unique_id_after = len(bcancer_ids)
print('patients with C50 in the period truncated problems table: '+str(unique_id_after))

Delete all records prior to 2012

In [ ]:
problemes = problemes[problemes.dat >= '2012-01-01']
facturacions = facturacions[facturacions.dat >= '2012-01-01']

Change all the datetime columns to number of months since birth, using the function *months_diff*.

In [ ]:
def months_diff(a, b):
    return round((a - b)/np.timedelta64(1, 'M'), 1)

In [ ]:
# Add birth date column to problemes and facturacions
problemes = pd.merge(problemes, pacients[['ID', 'dnaix']], on='ID', how='left')
facturacions = pd.merge(facturacions, pacients[['ID', 'dnaix']], on='ID', how='left')

# Change date columns to months since birth
pacients['sortida'] = months_diff(pacients.sortida, pacients.dnaix)
problemes['dat'] = months_diff(problemes.dat, problemes.dnaix)
problemes['dbaixa'] = months_diff(problemes.dbaixa, problemes.dnaix)
facturacions['dat'] = months_diff(facturacions.dat, facturacions.dnaix)

# Drop the column dnaix in all the tables, as it no longer gives additional information
pacients.drop(columns='dnaix', inplace=True)
problemes.drop(columns='dnaix', inplace=True)
facturacions.drop(columns='dnaix', inplace=True)

In [ ]:
pacients.head()

In [ ]:
problemes.head()

In [ ]:
facturacions.head()

Create auxiliar dataframe *df_cut* containing the first diagnosis of C50 for each patient, as well as the cutting date, given the number of months.

In [ ]:
df_cut = problemes[problemes.cod == 'C50'].groupby('ID')['dat'].min().reset_index()
df_cut.rename(columns={'dat': 'first_C50'}, inplace=True)
df_cut['cut_date'] = df_cut['first_C50'] + num_months
df_cut.head()

Drop patients who do not have enough information for the given number of months: active (A) or transferred (T) patients with *sortida* date before the cutting date

In [ ]:
# Check that the IDs set is the same in pacients and df_cut
assert set(pacients.ID) == set(df_cut.ID)

# Add cut_date column to pacients
pacients = pd.merge(pacients, df_cut[['ID', 'cut_date']], on='ID', how='left')

# Leave only dead patients or those with sortida later than cut_date
pacients = pacients[(pacients.situacio == 'D') | (pacients.sortida >= pacients.cut_date)]
ids_list = pacients.ID.unique()
problemes = problemes[problemes.ID.isin(ids_list)]
facturacions = facturacions[facturacions.ID.isin(ids_list)]

Change the state of the patients who died (D) after the cutting date to alive (A). Same for the patients transferred (T) after the cutting date.

In [ ]:
pacients.loc[pacients.situacio == 'T', 'situacio'] = 'A'
pacients.loc[(pacients.situacio == 'D') & (pacients.sortida >= pacients.cut_date), 'situacio'] = 'A'

Convert the categorical variable *situacio* to boolean. As ML algorithms work better with float values, the new column *is_dead* takes values 0.0 and 1.0

In [ ]:
pacients['is_dead'] = (pacients.situacio == 'D').astype(float)
pacients.drop(columns='situacio', inplace=True)

Drop information with dates posterior to the cutting date

In [ ]:
problemes = pd.merge(problemes, df_cut[['ID', 'cut_date']], on='ID', how='left')
facturacions = pd.merge(facturacions, df_cut[['ID', 'cut_date']], on='ID', how='left')

problemes = problemes[(problemes.dat <= problemes.cut_date)]
facturacions = facturacions[facturacions.dat <= facturacions.cut_date]

Fill in uninformed values in *dbaixa* with death date (if dead) or cutting date (if alive).

In [ ]:
problemes = pd.merge(problemes, pacients[['ID', 'is_dead', 'sortida']], on='ID', how='left')

filt_no_dbaixa = (problemes.dbaixa.isna())
filt_dead = (problemes.is_dead == 1.)
problemes.loc[filt_no_dbaixa & filt_dead, 'dbaixa'] = problemes[filt_no_dbaixa & filt_dead]['sortida']
problemes.loc[filt_no_dbaixa & ~filt_dead, 'dbaixa'] = problemes[filt_no_dbaixa & ~filt_dead]['cut_date']

Drop the columns added to the dataframes just for calculations (to avoid problems in future operations).

In [ ]:
pacients.drop(columns=['cut_date', 'sortida'], inplace=True)
problemes.drop(columns=['cut_date', 'is_dead', 'sortida'], inplace=True)
facturacions.drop(columns='cut_date', inplace=True)

Check the state of the tables after all the previous transformations

In [ ]:
pacients.head()

In [ ]:
problemes.head()

In [ ]:
facturacions.head()

## **Part II:** Aggregating, pivoting and merging into a master table

Remove duplicates of the pairs (ID, cod).
 - In **problemes** aggregate rows: minimum of *dat*, maximum of *dbaixa*, sum of *num_diag* (to count the number of the same diagnosis (ID, cod))
 - In **facturacions** aggregate rows: minimum and maximum of *dat*, sum of *env*

In [ ]:
problemes['num_diag'] = 1
problemes_gr = problemes.groupby(['ID', 'cod']).agg({'dat': 'min', 'dbaixa': 'max', 'num_diag': 'sum'}).reset_index()

problemes_gr.head()

In [ ]:
facturacions_gr = facturacions.groupby(['ID', 'cod']).agg({'dat': ['min', 'max'], 'env': 'sum'}).reset_index()
facturacions_gr.columns=['ID', 'cod', 'dat_min', 'dat_max', 'env_total']
facturacions_gr.head()

Check that there is only one row per patient and diagnosis/medication

In [ ]:
assert len(problemes_gr) == len(problemes[['ID', 'cod']].drop_duplicates())
assert len(facturacions_gr) == len(facturacions[['ID', 'cod']].drop_duplicates())

Create new columns which will be used for pivoting in the following steps. They contain the names of future dataframe columns

In [ ]:
problemes_gr['cod_diag'] = problemes_gr.cod+'_d'
problemes_gr['cod_recovery'] = problemes_gr.cod+'_r'
problemes_gr.head()

In [ ]:
facturacions_gr['cod_start_med'] = facturacions_gr.cod+'_s'
facturacions_gr['cod_end_med'] = facturacions_gr.cod+'_e'
facturacions_gr.head()

Pivot *problemes_gr* into three new auxiliary dataframes and merge them with *pacients*, creating a new dataframe *df_master*.

What are the values of the columns in each of the tables?

- In **problemes_aux1** the numbers represent the **number of times** the patient has been diagnosed with that illness.
- In **problemes_aux2** the dates (in number of months since birth) represent the **first time** the patient has been diagnosed with that illness.
- In **problemes_aux3** the dates (in number of months since birth) represent the **last recovery date** from that illness, or **death date** (if no recovery and death), or **cutting date** (if no recovery and alive).

In [ ]:
problemes_aux1 = problemes_gr.pivot_table(index='ID', columns='cod', values='num_diag').reset_index()
problemes_aux1.head()

In [ ]:
problemes_aux2 = problemes_gr.pivot(index='ID', columns='cod_diag', values='dat').reset_index()
problemes_aux2.head()

In [ ]:
problemes_aux3 = problemes_gr.pivot(index='ID', columns='cod_recovery', values='dbaixa').reset_index()
problemes_aux3.head()

**Estoy pensando que tal como lo has organizado puede que no fuera necesario quitar C50_r aunque no estoy seguro de si esta aportando algo realmente (miguel)** 

In [ ]:
df_master = pd.merge(pacients, problemes_aux1, how='left', on='ID')
df_master = pd.merge(df_master, problemes_aux2, how='left', on='ID')
df_master = pd.merge(df_master, problemes_aux3, how='left', on='ID')

df_master.head()

Pivot *problemes_gr* into three new auxiliary dataframes and merge them with *df_master*

What are the values of the columns in each of the tables?

- In **facturacions_aux1** the numbers represent the **number of medication boxes** the pacient has taken.
- In **facturacions_aux2** the dates represent the **first time** the patient has taken the medication.
- In **facturacions_aux3** the dates represent the **last time** the patient has taken the medication.

In [ ]:
facturacions_aux1 = facturacions_gr.pivot_table(index='ID', columns='cod', values='env_total').reset_index()
facturacions_aux1.head()

In [ ]:
facturacions_aux2 = facturacions_gr.pivot(index='ID', columns='cod_start_med', values='dat_min').reset_index()
facturacions_aux2.head()

In [ ]:
facturacions_aux3 = facturacions_gr.pivot(index='ID', columns='cod_end_med', values='dat_max').reset_index()
facturacions_aux3.head()

In [ ]:
df_master = pd.merge(df_master, facturacions_aux1, how='left', on='ID')
df_master = pd.merge(df_master, facturacions_aux2, how='left', on='ID')
df_master = pd.merge(df_master, facturacions_aux3, how='left', on='ID')

Drop the column ID, as it should not be used from now on

In [ ]:
df_master.drop(columns='ID', inplace=True)

The final master table

In [ ]:
df_master

## **Part III:** Save the master table to csv file

In [ ]:
df_master.to_csv(path + f'df_master_{num_months}.csv', index=False)